In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import load, Loader
import subprocess
from json import dump

## Parameters for Processing

In [2]:
output_path = Path("run3")
actual_path = output_path / "actual_time.yaml"
theoretical_path = output_path / "theoretical_time.yaml"
metadata_path = output_path / "metadata.yaml"
mem_folder_path = output_path / "memory_output"

data_dump_path = Path("processed_output")
output_fnames = ("actual_runtime.csv", "theoretical_runtime.csv", "bytes.csv", "all_fields.json")
MFLOPS = 870
FLOPS = MFLOPS * 10E6
MODELS = ["tf-necd", "tf-cod", "pytorch-qrcp", "pytorch-qr", "pytorch-svd", "pytorch-svddc", "sklearn-svddc"]

## Loading Metadata

In [3]:
with open(metadata_path, "r") as f:
    metadata = load(f, Loader=Loader)
    
metadata

{'dataset_shape': '1000000000 x 10',
 'failed_regs': [],
 'failed_regs_exceptions': [],
 'repeat': 10,
 'rows_in_experiment': [10,
  31,
  100,
  316,
  1000,
  3162,
  10000,
  31622,
  100000,
  316227,
  1000000,
  3162277,
  10000000,
  31622776,
  100000000,
  316227766,
  1000000000],
 'timer_method': 'process in nanoseconds'}

## Loading Actual Times

In [4]:
with open(actual_path, "r") as f:
    actual_times = load(f, Loader=Loader)
    
actual_times_dict = {reg_name: {row_count: np.mean([nanoseconds for rc, nanoseconds in actual_times if row_count == rc]) for row_count, _ in actual_times} for reg_name, actual_times in actual_times.items()}
actual_times_dict

{'pytorch-qr': {10: 420769.8,
  31: 340556.9,
  100: 356526.9,
  316: 348606.7,
  1000: 373437.7,
  3162: 428230.7,
  10000: 663926.7,
  31622: 1403934.6,
  100000: 3550642.8,
  316227: 11817954.2,
  1000000: 60295693.6,
  3162277: 210174951.6,
  10000000: 750175189.8,
  31622776: 2667246729.5,
  100000000: 8466698370.2,
  316227766: 28376192141.6,
  1000000000: 91116327422.5},
 'pytorch-qrcp': {10: 3996370.8,
  31: 364549.1,
  100: 403149.9,
  316: 385153.4,
  1000: 374110.9,
  3162: 452401.0,
  10000: 720017.0,
  31622: 1593570.5,
  100000: 3849959.6,
  316227: 13271848.5,
  1000000: 64038923.2,
  3162277: 229306137.9,
  10000000: 834783826.1,
  31622776: 2897131885.8,
  100000000: 8991294093.1,
  316227766: 30399364039.5,
  1000000000: 100164219748.7},
 'pytorch-svd': {10: 564496.9,
  31: 382512.7,
  100: 384810.8,
  316: 395247.9,
  1000: 416506.7,
  3162: 477696.8,
  10000: 681083.7,
  31622: 1389872.4,
  100000: 3398650.4,
  316227: 11729825.3,
  1000000: 60363017.2,
  3162277: 2

## Loading Theoretical Flop Counts

In [5]:
with open(theoretical_path, "r") as f:
    theoretical_flops = load(f, Loader=Loader)
    
theoretical_times_dict = {reg_name: {row_count: (flops / FLOPS) * 10E9 for row_count, flops in theoretical_flops} for reg_name, theoretical_flops in theoretical_flops.items()}
# theoretical_times_dict

## Loading / Processing Binary Memory Files

In [6]:
make_mempath = lambda reg_name, row_count, iter, ext: mem_folder_path / f"{'mem' if ext == 'bin' else 'memray-csv-mem'}_{reg_name}_{row_count}_{iter}.{ext}"

mem_dict = {}
for reg_name in MODELS:
    mem_dict[reg_name] = {}
    for row_count in metadata["rows_in_experiment"]:
        runs = []
        for iter in range(metadata["repeat"]):
            # use memray transform to actually create csv
            input_path = make_mempath(reg_name, row_count, iter, "bin")
            print(f"{input_path =}")
            subprocess.run(["memray", "transform", "csv", input_path, "-f"])
            
            # load csv normally, sum all mem usage
            output_path = make_mempath(reg_name, row_count, iter, "csv")
            single_mem_df = pd.read_csv(output_path, header=0, index_col=None)
            bytes_used = single_mem_df["size"].sum()
            runs.append(bytes_used)
            
        mem_dict[reg_name][row_count] = np.mean(runs)

input_path =PosixPath('run3/memory_output/mem_tf-necd_10_0.bin')

⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately 
reflect stack traces. Please use an interpreter built with debug symbols for 
best results. Check https://bloomberg.github.io/memray/native_mode.html for more
information regarding how memray resolves symbols.


Wrote run3/memory_output/memray-csv-mem_tf-necd_10_0.csv
input_path =PosixPath('run3/memory_output/mem_tf-necd_10_1.bin')

⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately 
reflect stack traces. Please use an interpreter built with debug symbols for 
best results. Check https://bloomberg.github.io/memray/native_mode.html for more
information regarding how memray resolves symbols.


Wrote run3/memory_output/memray-csv-mem_tf-necd_10_1.csv
input_path =PosixPath('run3/memory_outpu

## Aggregate and Output Data

In [7]:
json_prep_dict = {}
for reg_name in MODELS:
    json_prep_dict[reg_name] = {}
    for row_count in metadata["rows_in_experiment"]:
        json_prep_dict[reg_name][row_count] = {
            "real_runtime": actual_times_dict[reg_name][row_count],
            "theoretical_runtime": theoretical_times_dict[reg_name][row_count],
            "bytes": mem_dict[reg_name][row_count]
        }

df_real_runtime = pd.DataFrame(actual_times_dict)
df_theoretical_runtime = pd.DataFrame(theoretical_times_dict)
df_memory = pd.DataFrame(mem_dict)

data_dump_path.mkdir(exist_ok=True)

df_real_runtime.to_csv(data_dump_path / output_fnames[0], header=True, index=True)
df_theoretical_runtime.to_csv(data_dump_path / output_fnames[1], header=True, index=True)
df_memory.to_csv(data_dump_path / output_fnames[2], header=True, index=True)

with open(data_dump_path / output_fnames[3], "w") as f:
    dump(json_prep_dict, f)
    
print(f"All data has been dumped to: {data_dump_path}")

All data has been dumped to: processed_output
